## Dataset

In [4]:
import torch
from torch.utils.data import Dataset

In [9]:
class CustomDataset_jy(Dataset):
  def __init__(self, data, labels, transform=None):
    self.data = torch.tensor(data).float()
    # 작아서 이렇게 한번에 넣고 처리함
    # 만약, 일반적인 경우에서도 이렇게 하면 부하가 많이 가해져서 위험할  수도 있음
    self.labels = torch.tensor(labels).float()
    self.transform = None # transform 설정

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sample = self.data[idx]
    label = self.labels[idx]
    if self.transform: # instance별로 transform 적용
      sample = self.transform(sample)
    return sample, label

x = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
y = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

dataset = CustomDataset_jy(x, y)

In [15]:
i = iter(dataset)
print(next(i))
# print(i.__next__())

(tensor(0.5000), tensor(35.7000))


## DataLoader

In [19]:
from torch.utils.data import DataLoader

In [ ]:
data_loader = DataLoader(
    dataset,     # torch.utils.data.Dataset의 instance
    batch_size,  # batch의 샘플수
    shffule,     # boolean, 셔플링을 할지 여부(순서를 랜덤하게)
    num_workers, # 데이터로딩에 사용되는 sub-process의 수 (CPU의 core수를 넘으면 안됨.)
    pin_memory,  # boolean, GPU memory 영역을 예약할지 여부(pin).
    drop_last,   # boolean, 마지막 batch가 샘플의 수가 맞지 않을 경우 dorp할지 여부.
    collate_fn,  # callable, 샘플 리스트를 배치로 변환하는 함수
                 # None: 기본 collate_fn 사용 (텐서 자동 스택, 딕셔너리 처리 등)
                 # 커스텀 함수: 가변 길이 패딩, 배치 단위 전처리 등에 사용
    )

위에는 그냥 두기

In [20]:
data_loader = DataLoader(
    dataset,
    batch_size = 4,
    shuffle = True,
)

for batch_idx, (data, labels) in enumerate(data_loader):
    print(f'{batch_idx=}')
    print(f'{data.shape} | {data=}')
    print(f'{labels.shape} | {labels=}')

    # training ...

batch_idx=0
torch.Size([4]) | data=tensor([ 0.5000,  8.0000, 14.0000, 13.0000])
torch.Size([4]) | labels=tensor([35.7000, 48.9000, 55.9000, 60.4000])
batch_idx=1
torch.Size([4]) | data=tensor([ 6., 15., 11., 21.])
torch.Size([4]) | labels=tensor([48.4000, 58.2000, 56.3000, 68.4000])
batch_idx=2
torch.Size([3]) | data=tensor([ 3., -4., 28.])
torch.Size([3]) | labels=tensor([33.9000, 21.8000, 81.9000])


## 이 아래는 dataset을 list로 해서 load로 .....

In [25]:
class ListDataset_jy(Dataset):
    def __init__(self, transform=None):
        self.data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        self.labels = [1, 2, 3]
        self.transform = transform  # transform을 초기화할 때 인자로 받기

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:  # transform이 있을 때만 적용
            sample = self.transform(sample)
        return sample, label

In [27]:
dataset = ListDataset_jy()
print(dataset[0])

([1, 2, 3], 1)


In [28]:
for idx, i in enumerate(dataset):
  print(f"{idx = :02} | {i = }")

idx = 00 | i = ([1, 2, 3], 1)
idx = 01 | i = ([4, 5, 6], 2)
idx = 02 | i = ([7, 8, 9], 3)


In [29]:
dl = DataLoader(dataset, batch_size=2, )

In [30]:
for idx, i in enumerate(dl):
  print(i)

[[tensor([1, 4]), tensor([2, 5]), tensor([3, 6])], tensor([1, 2])]
[[tensor([7]), tensor([8]), tensor([9])], tensor([3])]


In [32]:
dl = DataLoader(dataset, batch_size=2, shuffle=False)

In [33]:
for idx, i in enumerate(dl):
  print(f"{idx = :02} | {i = }")

idx = 00 | i = [[tensor([1, 4]), tensor([2, 5]), tensor([3, 6])], tensor([1, 2])]
idx = 01 | i = [[tensor([7]), tensor([8]), tensor([9])], tensor([3])]
